# Tables Needed
## engine.transfer_event, engine.daily_conversion_rates, engine.account_derived_metadata, gemini.exchange_users.level/exchange_account_id, 

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import json
from os import listdir
from os.path import isfile, join
import re
import shutil
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# database variables
cred     = json.load(open('credential_zec.json'))
db1 = 'gemini'
db2 = 'engine'
db3 = 'wallet'
port     = '55432'
user1     = cred[db1]['username']
password1 = cred[db1]['password']
user2     = cred[db2]['username']
password2 = cred[db2]['password']
user3     = cred[db3]['username']
password3 = cred[db3]['password']
localhost = '127.0.0.1'

conn_gem = psycopg2.connect(host=localhost,
    port=port,
    user=user1,
    password=password1,
    dbname=db1)

conn_eng = psycopg2.connect(host=localhost,
    port=port,
    user=user2,
    password=password2,
    dbname=db2)

conn_wal = psycopg2.connect(host=localhost,
    port=port,
    user=user3,
    password=password3,
    dbname=db3)

def query(sql, c):
    return pd.read_sql_query(sql, c)

def query_file(sql_file, c):
    with open(sql_file, "r") as f:
        sql = f.read()
    return query(sql, c)

/Users/sqian/.pyenv/versions/3.7.0/envs/env_370/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


FileNotFoundError: [Errno 2] No such file or directory: 'credential_zec.json'

In [2]:
xfer_query = '''
SELECT 
  transfer_event.account_id,
  adm.user_or_account_name,
  date(transfer_event.tx_time),
  transfer_event.metadata::varchar,
	transfer_event.amount  AS "amount",
	COALESCE(SUM(CASE
           WHEN transfer_event.currency  = 'USD'
            THEN (transfer_event.amount )
           ELSE (transfer_event.amount ) * (daily_conversion_rates.price)
           END), 0) AS "amount_measure_usd"
FROM public.transfer_event  AS transfer_event
LEFT JOIN public.daily_conversion_rates  AS daily_conversion_rates ON (DATE(transfer_event.tx_time)) = (DATE(daily_conversion_rates.created )) and (CASE
      WHEN transfer_event.metadata::JSON #>> '{payload,$type}' IN ('OberonDepositedV1', 'OberonWithdrawnV1') THEN 'GUSD'
      ELSE transfer_event.currency
      END
) =  (CASE
          WHEN right(cast(daily_conversion_rates.trading_pair as  varchar(10)),3) = 'USD'
          THEN left(cast(daily_conversion_rates.trading_pair as  varchar(10)),3)
          END)
LEFT JOIN account_derived_metadata adm on adm.exchange_account_id = transfer_event.account_id
WHERE date(tx_time) between '2019-07-01' and '2019-08-01' AND
  currency = 'ZEC' AND
  type = 'Deposit'
GROUP BY 1,2,3,4,5
ORDER BY 2 DESC
'''

In [3]:
xfers = query(xfer_query, conn_eng)

In [4]:
accounts_list = (',').join([str(n) for n in xfers['account_id']])

In [10]:
risk_query = '''

WITH exchange_users AS (WITH distinct_account as (
            SELECT
                seu.id as distinct_exchange_user_id
                ,min(coalesce(exchange_account_id, sea.id)) as exchange_account_id
            FROM scrubbed_exchange_users seu
            LEFT OUTER JOIN user_account_group_roles uagr
              ON seu.id = uagr.exchange_user_id
            LEFT OUTER JOIN account_group ag
              ON ag.account_group_id = uagr.account_group_id
            LEFT OUTER JOIN user_account_roles uar
              ON seu.id = uar.exchange_user_id
            LEFT OUTER JOIN scrubbed_exchange_accounts sea
              ON sea.account_group_id = ag.account_group_id
            GROUP BY 1
            )

      SELECT
        seu.*
        ,exchange_account_id
      FROM scrubbed_exchange_users seu
      INNER JOIN distinct_account da
        ON seu.id = da.distinct_exchange_user_id
    )
        
SELECT
    exchange_account_id,
    level
FROM exchange_users
WHERE exchange_account_id IN ({})
'''.format(accounts_list)

In [11]:
risks = query(risk_query, conn_gem)

In [12]:
# tx_id = [json.loads(n)['reason']['outpoint']['txId'] for n in xfers.metadata]

jsons = []

for n in xfers.metadata:
    try:
        blob = json.loads(n)
        jsons.append(blob['reason']['outpoint']['txId'])
    except:
        jsons.append(blob['payload']['txHash'])

In [13]:
tx_ids_list = (', ').join(str("'")+n+str("'") for n in jsons)

In [14]:
xfers['tx_id'] = jsons

In [15]:
wallet_query = '''
SELECT
    tx_id,
    is_shielded
FROM nakamoto_relevants_zec
WHERE tx_id IN ({})
'''.format(tx_ids_list)

In [16]:
shielded = query(wallet_query, conn_wal)

In [17]:
xfers.head()
# names.head()
risks.head()
shielded.head()

,account_id,user_or_account_name,date,metadata,amount,amount_measure_usd,tx_id
0,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...
1,492918,zia Abhari,2019-07-02,"{""advanceId"":7220424450,""account"":492918,""amou...",0.001,0.133,ffd28ac06b603d4986871ead95266653a288c1b697e424...
2,492918,zia Abhari,2019-07-03,"{""advanceId"":7234184752,""account"":492918,""amou...",0.001,0.138,83c3160ffd6a113c2835051b1684064fd01562f572300f...
3,492918,zia Abhari,2019-07-04,"{""advanceId"":7251251058,""account"":492918,""amou...",0.001,0.142,e3d158fc1a65f60cc3c2f6497de6858ade982841451201...
4,492918,zia Abhari,2019-07-05,"{""advanceId"":7267359973,""account"":492918,""amou...",0.001,0.127,70a5d95f4511b09e23c96c015a11f09c343167cc454e3a...


,exchange_account_id,level
0,889667,EnteringPersonalInfo
1,22440,EnteringPersonalInfo
2,240114,LowRisk
3,882605,HighRisk
4,444031,LowRisk


,tx_id,is_shielded
0,819c31662bc444f7f23507da725d256056970c89ad2c4a...,False
1,c7e868cece179a8913aab2f22973dcd7250d3f571fdcc4...,False
2,b5cb8e326bec64bbd5d7c2ea05128b18fe3742bad4e7b4...,False
3,52600234053eb708239844b932eb8203aaad1960eb4f2d...,False
4,52600234053eb708239844b932eb8203aaad1960eb4f2d...,False


In [18]:
final_df = pd.merge(xfers, risks, how = 'left', left_on = 'account_id', right_on = 'exchange_account_id')
final_df = pd.merge(final_df, shielded, how = 'left', left_on = 'tx_id', right_on = 'tx_id')

In [19]:
final_df.head()

,account_id,user_or_account_name,date,metadata,amount,amount_measure_usd,tx_id,exchange_account_id,level,is_shielded
0,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...,492918.000,HighRisk,False
1,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...,492918.000,HighRisk,False
2,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...,492918.000,HighRisk,False
3,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...,492918.000,HighRisk,False
4,492918,zia Abhari,2019-07-01,"{""advanceId"":7199341204,""account"":492918,""amou...",0.001,0.127,e7c5b8f6a7c2614a068aff9620ab562b57b67331b14ca5...,492918.000,HighRisk,False


In [20]:
final_df = final_df[final_df['is_shielded'] == True]

In [21]:
final_df.shape

(230, 10)

In [22]:
final_df.groupby('account_id')['amount_measure_usd'].sum()

account_id
25059       25304.037
47085        1273.488
223785        456.478
247006         37.263
372147          5.466
409546        318.862
839955       5355.000
846275        610.822
851658      29468.646
855525          3.022
859243          1.597
882605        515.212
890287    1882109.407
901792       1951.500
908364          2.360
960608       2396.667
1289590    123761.541
Name: amount_measure_usd, dtype: float64

In [23]:
account_sums = final_df.groupby('account_id')['amount_measure_usd'].sum()

In [24]:
greater_than_10k = account_sums[account_sums.values > 10000].index

In [25]:
greater_than_10k

Int64Index([25059, 851658, 890287, 1289590], dtype='int64', name='account_id')

In [26]:
final_df = final_df[final_df['account_id'].isin(greater_than_10k)]

In [27]:
final_df.columns

Index(['account_id', 'user_or_account_name', 'date', 'metadata', 'amount',
       'amount_measure_usd', 'tx_id', 'exchange_account_id', 'level',
       'is_shielded'],
      dtype='object')

In [28]:
final_df = final_df.drop(columns = ['metadata', 'tx_id', 'exchange_account_id', 'is_shielded'])

In [29]:
final_df.to_csv('unshielded_zec_txs.csv', index = False)